<a href="https://colab.research.google.com/github/aasyngier/machine-learning/blob/main/ml_homework11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework Assignment: Understanding Deconvolution in Autoencoders**
---------------

In class, we worked with autoencoders built from multilayer perceptrons (MLPs). However, encoders are often constructed using convolutional architectures to better capture spatial patterns. In this assignment, you'll explore how the decoder can use deconvolutional (transposed convolution) layers to reverse and mirror the operations performed by the convolutional encoder.

While convolutional encoders are relatively well understood, **decoding (or upsampling) the compressed representation** using **deconvolutional layers** (also known as **transposed convolutions**) often raises questions.

This assignment is particularly relevant because deconvolution is a core component of the U-Net architecture, a prominent neural network used extensively in image segmentation tasks.

Your main objective is to deeply understand **how transposed convolution layers work**, and explain them in both words and visuals.


## **The Objective**

Understand and clearly explain how **transposed convolutions** work. Use 2D transposed convolutions and a small grid of 2D points as a working example.

You may need to do some additional reading to complete this assignment.

## **Tasks & Deliverables**

### 1. **Theory Exploration**

Using markdown cells in your Colab notebook, answer the following:

- What is a **transposed convolution**?
- How does it differ from a regular convolution?
- How does it upsample feature maps?
- What are **stride**, **padding**, and **kernel size**, and how do they influence the result in a transposed convolution?
- To earn full two points, your explanation must be detailed enough for a reader to reproduce the upsampling process step by step.


### 2. **Manual Diagram (by your hand, not a generated image)**

Carefully plan and draw **by hand** a diagram or a set of diagrams that:

- Explain the process of using **transposed convolution**.
- Use an example of a **small input grid of 2D points** which gets expanded into a larger output grid.
- Explain how stride, padding, and the kernel shape affect the result.
- Show intermediate steps of the operation, not just input and output.

**Scan or photograph your diagram(s)**, and upload it to your **GitHub repository** for this course.

Then embed it in your Colab notebook using markdown (you can find examples on *how to do it* in previous notebooks related to this class, e.g. the one on linear regression or the one on the MLP network).


### 3. **Publish on GitHub**  
   - Place the Colab notebook in your **GitHub repository** for this course.
   - In your repository’s **README**, add a **link** to the notebook and also include an **“Open in Colab”** badge at the top of the notebook so it can be launched directly from GitHub.

# **Task 1**


## **What is a transposed convolution?**

A transposed convolution is a learnable operation that expands a feature map instead of shrinking it. Algebraically it corresponds to the transpose of the matrix that carries out an ordinary convolution, so running data forward through this layer has the same effect as running data backward through a standard convolution. In practice the layer distributes each input value through the kernel and writes those weighted values into a larger output grid, accumulating any positions where different kernel placements overlap.

## **How does it differ from a regular convolution?**

A conventional convolution slides a kernel across an input image, combines the values inside each local window, and places a single number at the corresponding output location—this usually reduces the height and width of the data. A transposed convolution works in the opposite direction: each single input value is multiplied by the entire kernel and that block of numbers is added to an output canvas. Because the blocks are dropped at locations spaced by the stride, the output canvas ends up wider and taller. Padding is handled after the fact by trimming the outer rows and columns, whereas padding for a forward convolution is supplied before the multiply-and-sum step. In a neural network the forward convolution is typically used on the encoder side, while the transposed version appears in decoders and generators where up-sampling is required.

## **How does it upsample feature maps?**

1. Zero-insertion (stride expansion): Create an empty grid. Copy every input element into the grid and leave S−1 blank cells—conceptually zeros—between adjacent rows and columns. This turns an $H_{\mathrm{in}}\times W_{\mathrm{in}}$ tensor into $\bigl((H_{\mathrm{in}}-1)\,S+1\bigr)\times\bigl((W_{\mathrm{in}}-1)\,S+1\bigr)$.

2. Optional padding: Add P zero rows to the top and bottom and P zero columns to the left and right. Padding is applied after the zero-insertion because the layer’s goal is to crop—not extend—the final borders.

3. Unit-stride convolution with the kernel: Now perform an ordinary convolution whose stride is exactly 1. The kernel weights are the same learnable parameters you would use in a forward convolution; mathematically this operation multiplies by the transpose of the original convolution matrix.

4. Overlap accumulation: Whenever two kernel placements cover the same cell, their products are summed.
If S < K these overlaps are frequent and lead to smooth interpolation; if S = K the blocks never overlap and each input pixel simply becomes a non-overlapping patch.

5. Border trimming (implicit when P > 0): After the convolution, remove exactly P rows and columns from every edge. The remaining array has spatial dimensions given by $H_{\text{out}} = (H_{\text{in}} - 1)\,S - 2P + K,\quad
W_{\text{out}} = (W_{\text{in}} - 1)\,S - 2P + K.$


## **What are stride, padding, and kernel size, and how do they influence the result in a transposed convolution?**

- Kernel size K: K specifies how large each “stamp” of weights is when an input coefficient is projected onto the output grid. A larger kernel increases the receptive field of every output pixel and raises the additive overlap between neighbouring projections, often producing smoother results but also increasing computation and parameter count. It appears directly in the output-size formula as the final “+ K” term.

- Stride S: Stride controls the spacing between successive kernel origins before the unit-stride convolution. Every increment of S inserts (S−1) rows and columns of zeros, so spatial dimensions scale approximately by S. When S is smaller than K, the scattered patches overlap; when it equals K, patches touch but do not overlap; when it exceeds K, gaps appear and the output may exhibit checkerboard artefacts unless padding is adjusted.

- Padding P: In transposed convolutions, padding denotes how many rows/columns are cropped off after the scatter-and-convolve phase. Increasing P therefore reduces the final height and width by 2P. A non-zero P is also a common remedy for edge artefacts introduced by large strides.

# **Task 2**


In [2]:
from IPython.display import Image, display
display(Image(filename="transposed_convolution.jpg"))